### Introduction
This notebook assumes that metronome service is already running locally on default port 5000 and is ready to receive HTTP calls. It will not work if the service is not running.
I will present a series of time points and then try to predict whether there is a pattern of points repeating every 5 minutes. 

### Data generation
Firstly, I will generate the series of time points repeating every 5 minutes and then inject some additional random data points into it.

In [4]:
import pandas as pd 
import numpy as np 
import json 
import requests


data_range = pd.date_range("20/05/2020 12:00", "20/05/2020 16:00", freq="5min")
print(data_range)

DatetimeIndex(['2020-05-20 12:00:00', '2020-05-20 12:05:00',
               '2020-05-20 12:10:00', '2020-05-20 12:15:00',
               '2020-05-20 12:20:00', '2020-05-20 12:25:00',
               '2020-05-20 12:30:00', '2020-05-20 12:35:00',
               '2020-05-20 12:40:00', '2020-05-20 12:45:00',
               '2020-05-20 12:50:00', '2020-05-20 12:55:00',
               '2020-05-20 13:00:00', '2020-05-20 13:05:00',
               '2020-05-20 13:10:00', '2020-05-20 13:15:00',
               '2020-05-20 13:20:00', '2020-05-20 13:25:00',
               '2020-05-20 13:30:00', '2020-05-20 13:35:00',
               '2020-05-20 13:40:00', '2020-05-20 13:45:00',
               '2020-05-20 13:50:00', '2020-05-20 13:55:00',
               '2020-05-20 14:00:00', '2020-05-20 14:05:00',
               '2020-05-20 14:10:00', '2020-05-20 14:15:00',
               '2020-05-20 14:20:00', '2020-05-20 14:25:00',
               '2020-05-20 14:30:00', '2020-05-20 14:35:00',
               '2020-05-